<a href="https://colab.research.google.com/github/monasolgi/DeepLearning/blob/master/gcn_seg_Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
npz_data=np.load('/content/amazon_electronics_computers (1).npz')
npz_data.files

['adj_data',
 'adj_indices',
 'adj_indptr',
 'adj_shape',
 'attr_data',
 'attr_indices',
 'attr_indptr',
 'attr_shape',
 'labels',
 'class_names']

In [ ]:
#utils
import numpy as np
import scipy.sparse as sp
import torch
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer, normalize


def load_data():

 features = sp.csr_matrix((npz_data['attr_data'], npz_data['attr_indices'], npz_data['attr_indptr']),
                                        shape=npz_data['attr_shape'])
  
 adj= sp.coo_matrix(sp.csr_matrix((npz_data['adj_data'], npz_data['adj_indices'], npz_data['adj_indptr']),
                                   shape=npz_data['adj_shape']))
  

 labels=npz_data['labels']
 labels=binarize_labels(labels)

 adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
 adj = normalize(adj + sp.eye(adj.shape[0]))


 idx_train = range(10000)
 idx_val = range(10000, 11500)
 idx_test = range(11500, 13381)

 idx_train = torch.LongTensor(idx_train)
 idx_val = torch.LongTensor(idx_val)
 idx_test = torch.LongTensor(idx_test)

 features = torch.FloatTensor(np.array(features.todense()))
 labels = torch.LongTensor(np.where(labels)[1])
 adj = sparse_mx_to_torch_sparse_tensor(adj)

 
 return adj, features, labels, idx_train, idx_val, idx_test
 

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    #sum in every row 
    r_inv = np.power(rowsum, -1).flatten()
    # every sum to the power of -1 
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    #diagonal matrice 
    mx = r_mat_inv.dot(mx)
    return mx


def binarize_labels(labels, sparse_output=False, return_classes=False):
    if hasattr(labels[0], '__iter__'):  # labels[0] is iterable <=> multilabel format
        binarizer = MultiLabelBinarizer(sparse_output=sparse_output)
    else:
        binarizer = LabelBinarizer(sparse_output=sparse_output)
    label_matrix = binarizer.fit_transform(labels).astype(np.float32)
    return (label_matrix, binarizer.classes_) if return_classes else label_matrix
 

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)    


In [ ]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [ ]:
load_data()

(tensor(indices=tensor([[    0,   507,  6551,  ..., 13019, 13121, 13751],
                        [    0,     0,     0,  ..., 13751, 13751, 13751]]),
        values=tensor([0.2000, 0.0070, 0.0032,  ..., 0.0081, 0.0370, 0.0333]),
        size=(13752, 13752), nnz=505474, layout=torch.sparse_coo),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 1., 1., 0.]]),
 tensor([4, 4, 8,  ..., 8, 4, 0]),
 tensor([   0,    1,    2,  ..., 9997, 9998, 9999]),
 tensor([10000, 10001, 10002,  ..., 11497, 11498, 11499]),
 tensor([11500, 11501, 11502,  ..., 13378, 13379, 13380]))

In [ ]:
#layers

import math

import torch

from torch.nn.parameter import Parameter

#class parameter ,param haro cache mikone

from torch.nn.modules.module import Module


class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        
        #Sparse matrix multiplication=spmm
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ') '

In [ ]:
#models
import torch.nn as nn
import torch.nn.functional as F
#from layers import GraphConvolution


class GCN(nn.Module):
    def __init__(self, nfeat, nhid,nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        #self.gc2 = GraphConvolution(nhid, nhid2)
        #self.gc3 = GraphConvolution(nhid2, nclass)
        
        #adding a 3th layer didn't outperform
        
        self.dropout = dropout
        
    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        #can also use tanh,not very different result
        
        x = F.dropout(x, self.dropout, training=self.training)
        #return F.log_softmax(x, dim=1)
        #print(x.size())
    
        x = self.gc2(x, adj)
        
        #x = self.gc3(x, adj)
        #return x
       # print(x.size())
        return F.log_softmax(x, dim=1)


In [ ]:
#train

from __future__ import division
from __future__ import print_function
from torchsummary import summary

import time
import argparse
import numpy as np

#import torch
import torch.nn.functional as F
import torch.optim as optim


#from utils import load_data, accuracy
#from models import GCN

# Training settings

parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=16,
                    help='Number of hidden units.')

#parser.add_argument('--hidden2', type=int, default=8,
                    #help='Number of hidden units.')

parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')

#args = parser.parse_args()
#error midad khate bala,khate paein jaigozin shod

args = parser.parse_known_args()[0]

args.cuda = not args.no_cuda and torch.cuda.is_available()

np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data()

# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=args.hidden,
            nclass=10,
            dropout=args.dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=args.lr, weight_decay=args.weight_decay)

## to set cuda as your device if possible
##training on  GPU

if args.cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()
    
    ## train:adjust the weights on the neural network
    ## validation:used to minimize overfitting

def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    
    loss_train.backward()
    optimizer.step()

    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)
        loss_val = F.nll_loss(output[idx_val], labels[idx_val])
        acc_val = accuracy(output[idx_val], labels[idx_val])
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    ## Turn on evaluation mode which disables dropout.
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))


# Train model
t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

print(model)

#summary(model,[(13381,767),(13381,13381)])
# Testing
test()




Epoch: 0001 loss_train: 3.0389 acc_train: 0.0951 loss_val: 2.7131 acc_val: 0.1553 time: 0.1707s
Epoch: 0002 loss_train: 2.7622 acc_train: 0.1112 loss_val: 2.3863 acc_val: 0.0353 time: 0.1699s
Epoch: 0003 loss_train: 2.4400 acc_train: 0.0630 loss_val: 2.2620 acc_val: 0.3527 time: 0.1802s
Epoch: 0004 loss_train: 2.3125 acc_train: 0.3289 loss_val: 2.1631 acc_val: 0.3820 time: 0.1693s
Epoch: 0005 loss_train: 2.2410 acc_train: 0.3561 loss_val: 2.0329 acc_val: 0.3833 time: 0.1748s
Epoch: 0006 loss_train: 2.0842 acc_train: 0.3627 loss_val: 1.9107 acc_val: 0.3853 time: 0.1744s
Epoch: 0007 loss_train: 1.9773 acc_train: 0.3629 loss_val: 1.8489 acc_val: 0.3853 time: 0.1822s
Epoch: 0008 loss_train: 1.8919 acc_train: 0.3632 loss_val: 1.8067 acc_val: 0.3913 time: 0.1675s
Epoch: 0009 loss_train: 1.8323 acc_train: 0.3723 loss_val: 1.7782 acc_val: 0.4147 time: 0.1740s
Epoch: 0010 loss_train: 1.8231 acc_train: 0.3910 loss_val: 1.7574 acc_val: 0.4847 time: 0.1712s
Epoch: 0011 loss_train: 1.7823 acc_train